<a href="https://colab.research.google.com/github/xiborxu/IST462xibo/blob/main/IST462_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('./Crash_Reporting_-_Drivers_Data-2.csv')

In [ ]:
#url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSxw25lAmepqXjvvKOUWlaVo-3o3LdcmfOT-1ZoqQU2kdi79RPOAKX-713JFT7mUy7gmLKilTcFzA2n/pub?gid=762249656&single=true&output=csv"
#df = pd.read_csv(url)

In [ ]:
df.shape # find the number of rows and columns in our data frame
         # this shows we have 168,157 rows data

In [ ]:
df.columns #check the columns of dataframe

In [ ]:
df.isnull().sum() # find out the number of missing value (null value)
                  # for some columns with significant values loss, drop them

In [ ]:
# draw the graph by cases number reported by each agency
import matplotlib.pyplot as plt
value_counts = df['Agency Name'].value_counts() # Count the values of each agency name
plt.figure(figsize=(8, 6))
value_counts.plot(kind='bar', color='skyblue')
plt.xlabel('Agency Name')
plt.ylabel('Report Value Counts')
plt.title('Report Cases by Agency Name')
plt.show()

In [ ]:
# draw the graph to find out the missing cases of each agency
df_missing = df[df['Non-Motorist Substance Abuse'].isna()]
# a subset from df, selecting the rows whose 'Non-Motorist Substance Abuse' column is missing
missing_values_count = df_missing['Agency Name'].value_counts()
# count the number of agency, that will be the time that agency miss the data
plt.figure(figsize=(12, 7))
missing_values_count.plot(kind='bar', color='red')
plt.xlabel('Agency Name')
plt.ylabel('Number of Missing Values')
plt.title('Missing Values by Agency')
plt.show()

In [ ]:
columns_drop = ['Road Name','Cross-Street Name','Report Number', 'Local Case Number', 'Off-Road Description', 'Municipality', 'Related Non-Motorist','Drivers License State', 'Non-Motorist Substance Abuse', 'Person ID', 'Circumstance','Vehicle ID']
# choose the columns we believe is useless
df = df.drop(columns_drop, axis=1)
#drop the columns

In [ ]:
columns_to_fill_UNKNOWN = ['Driver Substance Abuse', 'Traffic Control','Weather','Vehicle Damage Extent','Equipment Problems','Surface Condition','Collision Type','Light','Vehicle First Impact Location','Vehicle Second Impact Location','Vehicle Movement','Vehicle Make','Vehicle Model','Vehicle Body Type']
df[columns_to_fill_UNKNOWN] = df[columns_to_fill_UNKNOWN].fillna('UNKNOWN')
# previous columns have missing values and "UNKNOWN", so we convert the missing value into UNKNOWN

columns_to_fill_Unknown = ['Route Type','Cross-Street Type','Vehicle Continuing Dir','Vehicle Going Dir']
df[columns_to_fill_Unknown] = df[columns_to_fill_Unknown].fillna('Unknown')
# same above, covert to Unknown

In [ ]:
df['Vehicle Make'] = df['Vehicle Make'].replace('TOYT', 'TOYOTA')
df['Vehicle Make'] = df['Vehicle Make'].replace('HOND', 'HONDA')
df['Vehicle Make'] = df['Vehicle Make'].replace('CHEV', 'CHEVROLET')
df['Vehicle Make'] = df['Vehicle Make'].replace('CHEVY', 'CHEVROLET')
# we find that there are different name for Vehicle make, such as CHEV, CHEVY for CHEVROLET
# We decide to convert to same name
# though our method cannot totally avoid the problem, we can ensure the top 5 is correct,and that's enough for our analysis

In [ ]:
df.isnull().sum()
# no null values

In [ ]:
df.columns

In [ ]:
df = df[df['Vehicle Damage Extent'] != 'UNKNOWN']
df = df[df['Vehicle Damage Extent'] != 'OTHER']
df.dropna(subset=['Vehicle Damage Extent'], inplace=True)
# the Vehicle Damage Extent is a key column
# We don't want any Unknown or other here; hence we drop them

In [ ]:
# Draw the graph to show distribution of each damage extend
plt.figure(figsize=(8, 6))
df['Vehicle Damage Extent'].value_counts().sort_index().plot(kind='bar', color='blue')
plt.title('Distribution of Categories')
plt.xlabel('Category')
plt.ylabel('Count')
plt.show()

In [ ]:
# Draw the graph for Crash Types
crash_type_counts = df['ACRS Report Type'].value_counts()
# create a subset from df and count the number of each value in ACRS Report Type
plt.figure(figsize=(8, 6))
plt.pie(crash_type_counts, labels=crash_type_counts.index,autopct='%1.1f%%', startangle=140)
# We draw a pie chart this time to find out the percentage of three types
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Distribution of Crash Types')
plt.show()

In [ ]:
# draw the graph of damage count by damaged level in different weather conditions
grouped = df.groupby(['Vehicle Damage Extent', 'Weather']).size().unstack(fill_value=0)
# groupby the Vehicle Damage Extent and weather and count the size
grouped.plot(kind='bar', stacked=True, figsize=(8, 6),colormap = 'tab10')
plt.xlabel('Weather')
plt.ylabel('Count')
plt.title('Damage Extent by Weather')
plt.legend(title='Damage Extent',bbox_to_anchor=(1.45, 1), loc='upper right')

In [ ]:
rain_data = df[df['Weather'] == 'RAINING']
snow_data = df[df['Weather'] == 'SNOW']
foggy_data = df[df['Weather'] == 'FOGGY']
# create new datasets to find cases in three conditions (rain, fog, snow)

In [ ]:
# draw the rain map
import plotly.express as px
# Plotting crash locations on an interactive map using Plotly
fig = px.scatter_mapbox(rain_data,
                        # database
                        lat="Latitude",
                        lon="Longitude",
                        mapbox_style= "carto-positron",
                        # set the visual style of background
                        color_discrete_sequence=["skyblue"], # Color for rain
                        title="Crash Locations by Weather Conditions",
                        zoom=8)
fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0})
fig.show()

In [ ]:
# draw the snow map
fig_snow = px.scatter_mapbox(snow_data,
                             lat="Latitude",
                             lon="Longitude",
                             mapbox_style="carto-positron",
                             color_discrete_sequence=["Black"], # Color for snow
                             title="Snow Accidents",
                             zoom=8)
fig_snow.update_layout(margin={"r":0, "t":0, "l":0, "b":0})
fig_snow.show()

In [ ]:
# draw the fog map
fig_foggy = px.scatter_mapbox(foggy_data,
                              lat="Latitude",
                              lon="Longitude",
                              mapbox_style="carto-positron",
                              color_discrete_sequence=["grey"], # Color for foggy
                              title="Foggy Accidents",
                              zoom=8)
fig_foggy.update_layout(margin={"r":0, "t":0, "l":0, "b":0})
fig_foggy.show()

In [ ]:
destroyed_df = df[df['Vehicle Damage Extent'] == 'DISABLING']
percentage_destroyed = (destroyed_df['Weather'].value_counts() / len(destroyed_df)) * 100
threshold = 5
percentage_destroyed_others = percentage_destroyed[percentage_destroyed >= threshold]
percentage_destroyed_others['Others'] = percentage_destroyed[percentage_destroyed < threshold].sum()
plt.figure(figsize=(8, 6))
plt.pie(percentage_destroyed_others, labels=percentage_destroyed_others.index, autopct='%1.1f%%', startangle=140)
plt.axis('equal')
plt.title('Weather Distribution for "Disabling" Damage Level')
plt.show()

In [ ]:
speed_limit_counts = df['Speed Limit'].value_counts()
top_5_speed_limits = speed_limit_counts.head(5)
top_5_speed_limits = top_5_speed_limits.sort_index()
plt.figure(figsize=(8, 6))
top_5_speed_limits.plot(kind='bar')
plt.xlabel('Speed Limit (MPH)')
plt.ylabel('Number of Accidents')
plt.title('Top 5 Speed Limits with Most Accidents')
plt.xticks(rotation='horizontal')
plt.show()

In [ ]:
# draw a pie chart to find out the surface condition in most frequent miles limit
filtered_df = df[df['Speed Limit'] == 35]
# choose all of accidents on road whose speed limit is 35m
surface_condition_percentage = (filtered_df['Surface Condition'].value_counts() / len(filtered_df)) * 100
# calculate the percentage of each road conditions
threshold = 2
small_parts = surface_condition_percentage[surface_condition_percentage < threshold]
surface_condition_percentage['Others'] = small_parts.sum()
surface_condition_percentage = surface_condition_percentage[surface_condition_percentage >= threshold]
'''
'''
# Create a pie chart for the specified speed limit and surface conditions
plt.figure(figsize=(8, 6))
plt.pie(surface_condition_percentage, labels=surface_condition_percentage.index, autopct='%1.1f%%', startangle=140)
plt.axis('equal')
# Equal aspect ratio ensures that pie is drawn as a circle.
plt.title(f'Surface Condition Distribution for 35 MPH')
plt.show()

In [ ]:
wet_surface_accidents = df[df['Surface Condition'] == 'WET']

# Plotting crash locations on an interactive map using Plotly
fig = px.scatter_mapbox(wet_surface_accidents,
                        lat="Latitude",
                        lon="Longitude",
                        mapbox_style="carto-positron",  # Choose map style
                        title="Accidents on Wet Surface Conditions",
                        zoom=8,  # Set initial zoom level
                        color_discrete_sequence=["skyblue"])  # Color for wet surface accidents

fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})  # Adjust layout margins
fig.show()

In [ ]:
df['Vehicle Make'].value_counts().head(5)
# counts the value of top 5 vehicle make

In [ ]:
df['Vehicle Year'].value_counts().head(5)
# counts the value of top 5 vehicle year

In [ ]:
def categorize_year(year):
    if year >= 2020:
        return '2020-2023'
    elif year >= 2017:
        return '2017-2020'
    elif year >= 2012:
        return '2012-2017'
    elif year >= 2008:
        return '2008-2012'
    elif year >= 2000:
        return '2000-2008'
    else:
        return '2000 and before'

df['Year Range'] = df['Vehicle Year'].apply(categorize_year)
# create a new columns containing the bin of year
selected_makes = ['TOYOTA', 'HONDA', 'FORD', 'CHEVROLET', 'NISSAN']
# select the top 5 vehicle make
filtered_df = df[df['Vehicle Make'].isin(selected_makes)]
grouped = filtered_df.groupby(['Vehicle Make', 'Year Range']).size().unstack(fill_value=0)
percentage_grouped = grouped.div(grouped.sum(axis=1), axis=0) * 100
ax = percentage_grouped.plot(kind='bar', stacked=True, figsize=(10, 6))
# draw the graph
plt.xlabel('Vehicle Make')
plt.ylabel('Percentage (%)')
plt.title('Year Range Distribution by Vehicle Make')
plt.legend(title='Year Range', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=0)

In [ ]:
top_5_brands = df['Vehicle Make'].value_counts().head(5).index.tolist()

# Filter the data for the top 5 brands
filtered_df = df[df['Vehicle Make'].isin(top_5_brands)]

# Group the filtered data by "Brand" and "Damage Extent" and count occurrences
grouped = filtered_df.groupby(['Vehicle Make', 'Vehicle Damage Extent']).size().unstack(fill_value=0)

# Calculate the percentage for each damage extent level within each brand
percentage_grouped = grouped.div(grouped.sum(axis=1), axis=0) * 100

# Create a stacked bar chart to visualize the percentage distribution
ax = percentage_grouped.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.xlabel('Brand')
plt.ylabel('Percentage (%)')
plt.title('Damage Extent Distribution for Top 5 Brands')
plt.legend(title='Damage Extent', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)

In [ ]:
fatal_crashes = df[df['ACRS Report Type'] == 'Fatal Crash']

# Plotting crash locations on an interactive map using Plotly
fig = px.scatter_mapbox(fatal_crashes,
                        lat="Latitude",
                        lon="Longitude",
                        mapbox_style="carto-positron",  # Choose map style
                        title="Locations of Fatal Crashes",
                        zoom=8,  # Set initial zoom level
                        color_discrete_sequence=["red"])  # Color for fatal crashes

fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})  # Adjust layout margins
fig.show()

In [ ]:
# Extracting the column related to traffic control mechanisms
traffic_control_data = df['Traffic Control']

# Counting the occurrences of crashes for each type of traffic control mechanism
traffic_control_counts = traffic_control_data.value_counts()

# Plotting a bar chart for traffic control mechanisms frequency
plt.figure(figsize=(8, 6))
traffic_control_counts.plot(kind='bar', color='skyblue')
plt.title('Frequency of Crashes Based on Traffic Control Mechanisms')
plt.xlabel('Traffic Control Mechanisms')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Convert 'Crash Date/Time' column to datetime format in the copied dataframe
df['Crash Date/Time'] = pd.to_datetime(df['Crash Date/Time'])

# Extract year from 'Crash Date/Time' column and create a new column 'Year'
df['Year'] = df['Crash Date/Time'].dt.year

# Group by 'Year' and count occurrences
yearly_crash_count = df['Year'].value_counts().sort_index()

# Plotting time-series data for yearly crash occurrences
plt.figure(figsize=(10, 6))
plt.plot(yearly_crash_count, marker='o', color='orange')

plt.title('Crash Occurrences Over Year')
plt.xlabel('Year')
plt.ylabel('Number of Crashes')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:


# Assuming 'df' contains the crash report dataset


# Convert 'Crash Date/Time' column to datetime format in the copied dataframe
df['Crash Date/Time'] = pd.to_datetime(df['Crash Date/Time'])


# Resample data to monthly frequency and count occurrences
monthly_crash_count = df.resample('M').size()  # Monthly frequency

# Plotting time-series data for monthly crash occurrences
plt.figure(figsize=(10, 6))
plt.plot(monthly_crash_count, label='Monthly', marker='o', color='skyblue')

plt.title('Crash Occurrences Over Month')
plt.xlabel('Time')
plt.ylabel('Number of Crashes')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
df.dtypes

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for column in df.select_dtypes(include=['object']).columns: # convert all object columns to integer
    df[column] = le.fit_transform(df[column])
df

In [ ]:
#corr = df.corr()
#corr.style.background_gradient(cmap = 'coolwarm') # draw the correlation graph

In [ ]:
from sklearn.model_selection import train_test_split
X = df[['Weather','Speed Limit','Surface Condition','Vehicle Year','Vehicle Make','Traffic Control','Year']]
y = df['Vehicle Damage Extent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [ ]:
importances = clf.feature_importances_
for idx, importance in enumerate(importances):
    print(f"Feature {idx + 1}: {importance:.4f}")
print('Feature 4 (Vehciel Year) is most important feature.')
print('Feature 5(Vehicle Make), Feature 7(Year), and Feature 2 (Speed Limit) are related important.')